In [11]:
import pandas as pd

# Sample DataFrame
df = pd.DataFrame({'Test': [None, 'Value1', None, 'Value2', None, None, 'Value3', None]})

# Fill blank values with the previous value
df['Test1'] = df['Test'].fillna(method='ffill')

print(df)


     Test   Test1
0    None    None
1  Value1  Value1
2    None  Value1
3  Value2  Value2
4    None  Value2
5    None  Value2
6  Value3  Value3
7    None  Value3


/var/folders/4m/j9kyjvln7ps4dkntd048fklh0000gn/T/ipykernel_2575/4125152650.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Test1'] = df['Test'].fillna(method='ffill')


In [1]:
import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

import pandas as pd
import numpy as np


In [ ]:
model = keras.models.Sequential()

model.add(layers.Dense(8,activation='relu'))
model.add(layers.Dense(64, activation='tanh'))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(1024, activation='tanh'))
model.add(layers.Dense(2028, activation='tanh'))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dense(32, activation='tanh'))
model.add(layers.Dense(1,activation='softmax'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_crossentropy'])

# Train the model
model.fit(X_train.fillna(0), y_train, epochs=50, batch_size=1000, validation_split=0.1, verbose=1)


word_dictionary = {}

for i in column_data['CLEAN']:
    word = i.split()
    for i in word:
        try:
            word_dictionary[i] +=1
        except:
            word_dictionary[i] =1

In [ ]:

def clean_string(string,remove_chars=['+','-',"(",")",'/','*']):
    
    '''
    Function to clean some of the punctuation which appears in Column Headers
    
    '''
    new_string = ""
    for char in string:
        if char not in remove_chars:
            new_string +=char
    return new_string

def create_heatmap(df,column_name='',corr_value=.1,figsize=(20,15)):
    
    sns.set(style='white')
    
    # View column with Abbreviated title or full. Abbreviated displays nicer.
    corr = df.corr()
    
    if len(column_name)!=0:
        corr = corr[[column_name]]
        corr = corr[abs(corr[column_name])>corr_value]
    
    mask= np.zeros_like(corr,dtype=bool)
    mask[np.triu_indices_from(mask)]=True
    f,ax = plt.subplots(figsize=figsize)
    cmap = sns.diverging_palette(220,10,as_cmap=True)
    sns.heatmap(corr,mask=mask,cmap=cmap,vmax=.3,center=0,square=True,linewidths=.5)
    
    plt.title('Heat Map of Correlation')
    plt.show()
    
    
def create_column_inclusion_review(df,
                               columns,
                               column_type_df='',
                               decile_review_df=''):
    
    df = df[columns].copy()
    
    temp_df = review_dataset_dict(df)
    
    if len(column_type_df)!=0:
        temp_df = temp_df.merge(column_type_df,on='Financial Ratio',how='left')
    if len(decile_review_df)!=0:
        temp_df = temp_df.merge(decile_review_df,on='Financial Ratio',how='left')
    
    return temp_df


def variable_review(df,
                    column_name,
                    og_column,
                    column_inclusion_review_df,
                    corr_weight=.15):
        
    print(column_inclusion_review_df[column_inclusion_review_df['Financial Ratio']==column_name].T)
    print(f"\n")
    
    create_heatmap(df[og_columns],column_name)
    create_decile(df,column_name)
    
    print('Top 20 Records')
    print(df.sort_values(column_name)[[column_name,'BANKRUPTCY_FLAG']].tail(20))
    
    print('Bottom 20 Records')
    print(df.sort_values(column_name)[[column_name,'BANKRUPTCY_FLAG']].head(20))
    
    return df[df[column_name].isnull()].T

# Process for reviewing Non Tier 1 Elements.

def review_single_variable_manully(df,
                                   column_name,
                                   baseline_columns,
                                   column_inclusion_review_df,
                                   og_columns,
                                   export_to_excel=0):
        
    import datetime
    now = datetime.datetime.now()
    
    # Currently Included Columns for Simple Reference
    print("Columns Currently In Scope:")
    for included in baseline_columns:
        print(f"{included}\n")
    
    # EDA
    variable_review(df,column_name,og_columns,column_inclusion_review_df)    
    print(f"Number of Null Records with Bankrupcy Flag Yes: {df[df[column_name].isnull()]['BANKRUPTCY_FLAG'].sum()}")
    
    # Review Questions
    blank_or_remove = input('Remove Null Records/ Zero Null Records/ Exit Loop (remove/zero/exit)')
    include_in_model = input('Subjective Belief as to whether variable should be included in model (include/exclude)')
    negative_value = input('Remove, Zero or Leave Negative Values (remove/zero/ignore)')
    decision_logic = input('Please Provide Comment on Decision for Archival Reference')
        
    record_df =  pd.DataFrame([blank_or_remove.lower(),include_in_model.lower(),negative_value.lower(),decision_logic.lower(),now],index=['Null Record Approach','Baseline V2 Model Inclusion',"Negative Valuation",'Archival Decisioning','Extract Time'],columns=[column_name]).T.reset_index().rename(columns={'index':'Financial Ratio'})
    
    if export_to_excel==1:
        clean_column_name = clean_string(column_name,remove_chars=['+','-',"(",")",'/','*']).replace(" ","_")
        record_df.to_excel(f"manual_review/manual_review_{clean_column_name}_{now.strftime('%d%m%y%h%m%s')}.xlsx",index=False)
            
    return record_df

def read_files_in_folder(folder_location,file_type='*',import_df=0):
    
    files_ = os.listdir(folder_location)
    
    if file_type =='*':
        files_desired = files_.copy()
    else:
        files_desired = [x for x in files_ if x.find('xlsx')>-1]
        
    if import_df ==1:
        final_df = pd.DataFrame()
        if file_type =='xlsx':
            pd_read = pd.read_excel
        elif file_type =='csv':
            pd_read = pd.read_csv      
        else:
            print('Update Function')              
        for file in files_desired:
            final_df = pd.concat([final_df,pd_read(f"{folder_location}/{file}")])
            
        return final_df
         
    return files_desired



def build_binary_classification_model(input_dim, 
                                      hidden_layer_sizes,
                                      activation, 
                                      optimizer,
                                      learning_rate,
                                      metrics):

    """Build a binary classification model using Keras.

      Args:
        input_dim: Number of features in the input data.
        hidden_layer_sizes: A list with the number of units in each hidden layer.
        activation: The activation function to use for the hidden layers.
        optimizer: The optimizer
        learning_rate: The desired learning rate for the optimizer.

      Returns:
        model: A tf.keras model.
    """
    # Instantiate Model
    model = keras.models.Sequential()

    # Add Input Layer
    model.add(layers.InputLayer(input_shape=(input_dim,)))

    # Add Hidden Layers
    for nodes in hidden_layer_sizes:
        model.add(layers.Dense(units=nodes, activation=activation))

    # Add Output Layer
    model.add(layers.Dense(units=1, activation='sigmoid'))

    # Configure optimizer and compile the model
    if optimizer == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        optimizer = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)

    return model


def train_model(X,
                y,
                input_dim,
                metrics,
                hidden_layer_sizes,
                activation, 
                optimizer,
                learning_rate,
                batch_size,
                num_epochs,
                validation_split,
                verbose=0):
                       

    # Build the model.
    model = build_binary_classification_model(input_dim=input_dim,
                                              hidden_layer_sizes=hidden_layer_sizes,
                                              activation=activation, 
                                              optimizer=optimizer,
                                              learning_rate=learning_rate,
                                              metrics=metrics)
    
    print(model.summary())     
                        
    # Train the model.
    history = model.fit(x=X,
                        y=y,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_split=validation_split,
                        verbose=verbose)

    # Retrieve the training metrics (after each train epoch) and the final test
    # accuracy.
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    plt.figure(figsize=(15,5))
    plt.plot(train_accuracy, label='train_accuracy')
    plt.plot(val_accuracy, label='validation accuracy')
    plt.xticks(range(num_epochs))
    plt.xlabel('Train epochs')
    plt.ylim(0, 1)
    plt.legend()
    plt.show()
    
 
    return history,model

def create_balanced_dataset(X,y,observations=0,column_name='BANKRUPTCY_FLAG'):
    
    '''
    Function to take observations and labels, combine them together and then select a even numher of random examples
    
    X - X_Test or X_Training
    y - y_test or y_training
    observation - Number of records from both Binary On and Binary Off Column
    column_name - Name of Binary Column to filter
    
    
    Used Default Value for purposes of reducing typing in code and given function created for Project Exclusively.
    
    '''
    
    # If length of observations is not defined, create a even 50/ 50 dataset
    if observations == 0:
        observations = y[column_name].sum()
        
    if observations>y[column_name].sum():
        observations = y[column_name].sum()
        
    temp_df = pd.concat([X,y],axis=1).copy()
    
    df1 = temp_df[temp_df[column_name]==1].sample(observations).copy()
    df2 = temp_df[temp_df[column_name]==0].sample(observations).copy()
    
    final_df = pd.concat([df1,df2])
    final_df = final_df.sample(frac=1)
    
    X = final_df.drop(column_name,axis=1)
    y = final_df[[column_name]]
    
    
    def train_neural_network(X,
                y,
                input_dim,
                metrics,
                hidden_layer_sizes,
                activation, 
                optimizer,
                learning_rate,
                batch_size,
                num_epochs,
                validation_split,
                verbose=0):
                       

    # Build the model.
    model = build_binary_classification_model(input_dim=input_dim,
                                              hidden_layer_sizes=hidden_layer_sizes,
                                              activation=activation, 
                                              optimizer=optimizer,
                                              learning_rate=learning_rate,
                                              metrics=metrics)
    
    print(model.summary())     
                        
    # Train the model.
    history = model.fit(x=X,
                        y=y,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_split=validation_split,
                        verbose=verbose)

    # Retrieve the training metrics (after each train epoch) and the final test
    # accuracy.
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    plt.figure(figsize=(15,5))
    plt.plot(train_accuracy, label='train_accuracy')
    plt.plot(val_accuracy, label='validation accuracy')
    plt.xticks(range(num_epochs))
    plt.xlabel('Train epochs')
    plt.ylim(0, 1)
    plt.legend()
    plt.show()
    
 
    return history,model





def knn_model_creation(X_df,
                       y_df,
                       neighbors=3):    
    model = KNeighborsClassifier(n_neighbors=neighbors)
    model.fit(X_df,y_df)    
    return model

def logistic_regression_creation(X_df,
                                 y_df,
                                 solver='liblinear'):
    '''
    ['liblinear','newton-cg','lbfgs','sag','saga']
    
    for i in [10,20,30]:
        temp_df = (y_probabilities >= i/100).astype(int)
        knn_df = pd.concat([knn_df,pd.DataFrame(temp_df,columns=[f'LR_{str(i)}p'])],axis=1)
        
        
    '''
    model = LogisticRegression(solver='liblinear')
    model.fit(X_df,y_df)
    return model


def random_forest_classifier(X_df,
                             y_df):
    '''
    important_features_rf = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False).head(10)
    
    
    '''
    model = RandomForestClassifier()
    model.fit(X_df,y_df)
    return model

def neural_network(X_df,
                   y_df,
                   input_dim, 
                   hidden_layer_sizes,
                   activation,
                   optimizer,
                   learning_rate,
                   metrics,
                   verbose=0):

    """Build a binary classification model using Keras.

      Args:
        input_dim: Number of features in the input data.
        hidden_layer_sizes: A list with the number of units in each hidden layer.
        activation: The activation function to use for the hidden layers.
        optimizer: The optimizer
        learning_rate: The desired learning rate for the optimizer.

      Returns:
        model: A tf.keras model.
    """
    # Instantiate Model
    model = keras.models.Sequential()

    # Add Input Layer
    model.add(layers.InputLayer(input_shape=(input_dim,)))

    # Add Hidden Layers
    for nodes in hidden_layer_sizes:
        model.add(layers.Dense(units=nodes, activation=activation))

    # Add Output Layer
    model.add(layers.Dense(units=1, activation='sigmoid'))

    # Configure optimizer and compile the model
    if optimizer == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        optimizer = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)

    return model
34/22:
    # Where am I going to take out the Validation Data.
    # Where am I going to do the Standardization of Data?
    # Where am I going to take out the testing data.



    Takes a Dataframe, which includes Training Data (which includes Label), 
    

    # Step 1 Scale Data
if scaling_method.lower() == 'standard_scalar':
    X = apply_standard_scaler(X_df,X_df.columns.values)
elif scaling_method.lower()=='min_max':
    X = apply_min_max_scaler(X_df,X_df.columns.values)

    if bankrupcy_observations == 'entire_dataset':
a        pass
    elif bankrupcy_observations == 'all_bankrupcies':
        X,y = create_balanced_dataset(X,y)
    else:
        X,y = create_balanced_dataset(X,y,bankrupcy_observations)
34/23:
def create_model(X_df,
                 y_df,model,
                 scaling_method = 'standard_scalar',
                 learning_rate=.05,
                 model_parameters={}):
    '''
    
    Function to generate ML Model, based on model type as defined in Parameters.
    Below values for illustration purposes, can utilize dictionary to update as desired.

    if model is neural_network, 
    model_parameters = {'hidden_layer_sizes':[8,16,32,64],
                                          'activation':'relu',
                                          'optimizer':'adam',
                                          'learning_rate':.05,
                                          'metrics':['accuracy', keras.metrics.Precision(),keras.metrics.Recall()]}
    if model Logistic Regression,
    model_parameters = {'solver':'liblinear'}

    if model knn,
    model_parameters = {'n_neighbors':3}

    if model random_forsest
    model_parameters = {}

    '''

    if model == 'neural_network':
        model = neural_network(X_df,
                               y_df,
                               input_dim=len(X.columns.values), 
                               hidden_layer_sizes= model_parameters['hidden_layer_sizes'],
                               activation=model_parameters['activation'], 
                               optimizer=model_parameters['optimizer'],
                               learning_rate=model_parameters['learning_rate'],
                               metrics=model_parameters['metrics'])

    elif model == 'logistic_regression':
        model = logistic_regression_creation(X_df,
                                             y_df,
                                             solver=model_parameters['solver'])
    elif model == 'random_forest':
        model = random_forest_classifier(X_df,
                                         y_df)
    elif model == 'knn':
        model = knn_model_creation(X_df,
                                   y_df,
                                   neighbors=model_parameters['n_neighbors'])
    return model

def generate_predicition(X_test_df,
                         y_test_df,
                         model,
                         py_model,
                         model_parameters):

    if model == 'neural_network':
        history = py_model.fit(x=X_test_df,
                            y=y_test_df,
                            batch_size=model_parameters['batch_size'],
                            epochs=model_parameters['num_epochs'])

        acc = pd.DataFrame(history.history['accuracy'],columns=["Training Accuracy"])
        loss = pd.DataFrame(history.history['loss'],columns=["Training Loss"])
        prec_str = columns=[x for x in history.history.keys() if x.find('pre')!=-1][0]
        recall_str = columns=[x for x in history.history.keys() if x.find('reca')!=-1][0]
        precision = pd.DataFrame(history.history[prec_str],columns=[prec_str]).rename(columns={prec_str:'Precision'})
        recall = pd.DataFrame(history.history[recall_str],columns=[recall_str]).rename(columns={recall_str:'Recall'})

        model_results_df = pd.concat([model_results_df,pd.DataFrame([x for x in range(0,len(model_results_df))],columns=['Epoch Number'])],axis=1)
        model_results_df['Dataset'] = 'To Be Determined'
        model_results_df['Model'] = model
        model_results_df['activation'] = model_parameters['activation']
        model_results_df['optimizer'] = model_parameters['optimizer']
        model_results_df['learning_rate'] = model_parameters['learning_rate']
        model_results_df['batch_size'] = model_parameters['batch_size']
        model_results_df['epochs'] = model_parameters['num_epochs']
        model_results_df['hidden_layer_sizes'] = text_manipulation(model_parameters['hidden_layer_sizes'])
        #model_results_df['bankrupt_observations'] = bankrupcy_observations
        #model_results_df['total_observations_read'] = len(X)
        return model_results_df
    
    elif model == 'knn':
        pred = py_model.predict(X_test_df)
        pred_df = pd.DataFrame(pred,columns=['Predicitions'])
        final_df = pd.concat([y_test_df.reset_index(drop=True),pred_df],axis=1)
        return final_df
    
    else:
        pred = py_model.predict(X_test_df,y_test_df)
        pred_df = pd.DataFrame(pred,columns=['Predicitions'])
        final_df = pd.concat([y_test_df.reset_index(drop=True),pred_df],axis=1)
        return final_df
    

    # else:
    #     predicition = py_model.predict(X_test)
    #     temp_x = pd.DataFrame(predicition,columns=[column_name])
    #     if df == "":
    #         return pd.concat([y_test_df,temp_df],axis=1)
    #     else:
    #         return pd.concat([df,temp_df],axis=1)
34/24:
X = dataset_dictionary['All Ratios Cleaned']
y = dataset_dictionary['labels']

model='logistic_regression'
model_parameters = {'solver':'liblinear'}

lr_  = create_model(X,
                      y,
                      model=model,
                      model_parameters=model_parameters)

model = 'knn'
model_parameters = {'n_neighbors':3}

knn_ = create_model(X,
                      y,
                      model=model,
                      model_parameters=model_parameters)
34/25: knn_.predict(X)
34/26: X
34/27: X_train, X_test, y_train, y_test  = train_test_split(X,y,test_size=0.3, random_state=15)
34/28:
X = dataset_dictionary['All Ratios Cleaned']
y = dataset_dictionary['labels']

X_train, X_test, y_train, y_test  = train_test_split(X,y,test_size=0.3, random_state=15)


model='logistic_regression'
model_parameters = {'solver':'liblinear'}

lr_  = create_model(X_train,
                      y_train,
                      model=model,
                      model_parameters=model_parameters)

model = 'knn'
model_parameters = {'n_neighbors':3}

knn_ = create_model(X_train,
                      y_train,
                      model=model,
                      model_parameters=model_parameters)
34/29: lr_.n_iter_
34/30:
X = dataset_dictionary['All Ratios Cleaned']
y = dataset_dictionary['labels']

X_train, X_test, y_train, y_test  = train_test_split(X,y,test_size=0.3, random_state=15)


model='logistic_regression'
model_parameters = {'solver':'lbfgs'}

lr_  = create_model(X_train,
                      y_train,
                      model=model,
                      model_parameters=model_parameters)

model = 'knn'
model_parameters = {'n_neighbors':3}

knn_ = create_model(X_train,
                      y_train,
                      model=model,
                      model_parameters=model_parameters)
34/31:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


def knn_model_creation(X_df,
                       y_df,
                       neighbors=3):    
    model = KNeighborsClassifier(n_neighbors=neighbors)
    model.fit(X_df,y_df)    
    return model

def logistic_regression_creation(X_df,
                                 y_df,
                                 max_iter=100,
                                 solver='liblinear'):
    '''
    ['liblinear','newton-cg','lbfgs','sag','saga']
    
    for i in [10,20,30]:
        temp_df = (y_probabilities >= i/100).astype(int)
        knn_df = pd.concat([knn_df,pd.DataFrame(temp_df,columns=[f'LR_{str(i)}p'])],axis=1)
        
        
    '''
    model = LogisticRegression(solver='liblinear',max_iter=max_iter)
    model.fit(X_df,y_df)
    return model


def random_forest_classifier(X_df,
                             y_df):
    '''
    important_features_rf = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False).head(10)
    
    
    '''
    model = RandomForestClassifier()
    model.fit(X_df,y_df)
    return model

def neural_network(X_df,
                   y_df,
                   input_dim, 
                   hidden_layer_sizes,
                   activation,
                   optimizer,
                   learning_rate,
                   metrics,
                   verbose=0):

    """Build a binary classification model using Keras.

      Args:
        input_dim: Number of features in the input data.
        hidden_layer_sizes: A list with the number of units in each hidden layer.
        activation: The activation function to use for the hidden layers.
        optimizer: The optimizer
        learning_rate: The desired learning rate for the optimizer.

      Returns:
        model: A tf.keras model.
    """
    # Instantiate Model
    model = keras.models.Sequential()

    # Add Input Layer
    model.add(layers.InputLayer(input_shape=(input_dim,)))

    # Add Hidden Layers
    for nodes in hidden_layer_sizes:
        model.add(layers.Dense(units=nodes, activation=activation))

    # Add Output Layer
    model.add(layers.Dense(units=1, activation='sigmoid'))

    # Configure optimizer and compile the model
    if optimizer == 'sgd':
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        optimizer = keras.optimizers.legacy.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)

    return model
34/32:
def create_model(X_df,
                 y_df,model,
                 scaling_method = 'standard_scalar',
                 learning_rate=.05,
                 model_parameters={}):
    '''
    
    Function to generate ML Model, based on model type as defined in Parameters.
    Below values for illustration purposes, can utilize dictionary to update as desired.

    if model is neural_network, 
    model_parameters = {'hidden_layer_sizes':[8,16,32,64],
                                          'activation':'relu',
                                          'optimizer':'adam',
                                          'learning_rate':.05,
                                          'metrics':['accuracy', keras.metrics.Precision(),keras.metrics.Recall()]}
    if model Logistic Regression,
    model_parameters = {'solver':'liblinear','max_iter':200}

    if model knn,
    model_parameters = {'n_neighbors':3}

    if model random_forsest
    model_parameters = {}

    '''

    if model == 'neural_network':
        model = neural_network(X_df,
                               y_df,
                               input_dim=len(X.columns.values), 
                               hidden_layer_sizes= model_parameters['hidden_layer_sizes'],
                               activation=model_parameters['activation'], 
                               optimizer=model_parameters['optimizer'],
                               learning_rate=model_parameters['learning_rate'],
                               metrics=model_parameters['metrics'])

    elif model == 'logistic_regression':
        model = logistic_regression_creation(X_df,
                                             y_df,
                                             max_iter=model_parameters['max_iter'],
                                             solver=model_parameters['solver'])
    elif model == 'random_forest':
        model = random_forest_classifier(X_df,
                                         y_df)
    elif model == 'knn':
        model = knn_model_creation(X_df,
                                   y_df,
                                   neighbors=model_parameters['n_neighbors'])
    return model
    

In [ ]:
import pandas as pd
url = 'https://en-wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory'
pd.read_html(url)

import pandas as pd
import requests

# Correct Wikipedia URL
url = 'https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory'

# Set User-Agent to mimic a real browser
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

# Make the request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML tables
    tables = pd.read_html(response.text)

    # Print the number of tables found
    print(f"Total tables found: {len(tables)}")

    # Show the first table
    print(tables[0].head())
else:
    print(f"Failed to fetch page, status code: {response.status_code}")


In [1]:
%history -g -f history.txt

In [ ]:
import pandas as pd

pd.DataFrame(['We Design Everything','We Design, You Watch','We Design, You Advise','We Design, You Help','You Design, We Help','You Design, We Advise','You Design, We Watch','You Design Everything'],[500,800,1000,1500,2000,3500,5000,8000],columns=['Price List']).reset_index().rename(columns={'Price List':"Service Offered",'index':"Price"})[['Service Offered','Price']]

![Alt Text](https://raw.githubusercontent.com/derek-dewald/d_functions/main/images/Consulting_Fees.png)


| Dataset Type   | Fits in RAM? | Fits on Local Disk? |
|---------------|-------------|---------------------|
| Small dataset | Yes         | Yes                 |
| Medium dataset | No         | Yes                 |
| Big dataset   | No         | No                  |

In [ ]:
! conda install numpy

In [ ]:
import pandas as pd
import datetime

df = pd.read_csv('DKnowledgeSheet.csv')

def column_reminder(df,columns=[]):
    if len(columns)==0:
        columns = df.columns
        
    for column in columns:
        print(df[column].value_counts())

def UpdateDictionary(definition,
                     overwrite_all=0,
                     description="",
                     command_code="",
                     classification="",
                     category="",
                     sub_category=""):
    
    df = pd.read_csv('DKnowledgeSheet.csv')
    
    df['Definition'] = df['Definition'].fillna("").apply(lambda x:x.lower())
    
    temp_df = df[df['Definition'].fillna("").str.contains(definition.lower())]

    if overwrite_all==1:

        if len(temp_df)>1:
            confirm = input('Are you Certain you wish to proceed? You will be deleting multiple records.')
            if confirm == 'Yes':
                pass
            else:
                return print("Manually Ended")
            
        df = df[~df['Definition'].fillna("").str.contains(definition.lower())]
        new_record = {'Definition': [definition],
                      'Description':[description],
                      'Command/ Code':[command_code],
                      'Classification':[classification],
                      'Category':[category],
                      'Sub Category':[sub_category]}
    
        updated_df = pd.concat([df,pd.DataFrame(new_record)]).reset_index(drop=True)
        df.to_csv(f"archive//archive_{datetime.datetime.now().strftime('%d-%b-%y')}.csv",index=False)
        updated_df.to_csv('DKnowledgeSheet.csv',index=False)
        print("Updated Dictionary")
    
        return updated_df
        
    elif len(temp_df)>1:
        return temp_df
    
    elif (len(temp_df)==1)&((len(description)>0)|(len(command_code)>0)|(len(classification)>0)|(len(category)>0)):
        value = temp_df['Definition'].item()    
        if len(description)>0:
            df.loc[df['Definition']==value,'Description']=description
        if len(command_code)>0:
            df.loc[df['Definition']==value,'Command/Code']=command_code        
        if len(classification)>0:
            df.loc[df['Definition']==value,'Classification']=classification
        if len(category)>0:
            df.loc[df['Definition']==value,'Category']=category
        if len(sub_category)>0:
            df.loc[df['Definition']==value,'Sub Category']=sub_category
            
        return df
    elif len(temp_df)==1:
        return temp_df
    elif (len(temp_df)==0)&(len(description)==0)&(len(command_code)==0)&(len(classification)==0)&(len(category)==0):
        print('No Records Observed, Nothing to Update')
                                                                            
    else:
        new_record = {'Definition': [definition],
                      'Description':[description],
                      'Command/ Code':[command_code],
                      'Classification':[classification],
                      'Category':[category],
                      'Sub Category':[sub_category]}
                     
        updated_df = pd.concat([df,pd.DataFrame(new_record)]).reset_index(drop=True)
        df.to_csv(f"archive//archive_{datetime.datetime.now().strftime('%d-%b-%y')}.csv",index=False)
        updated_df.to_csv('DKnowledgeSheet.csv',index=False)
        print("Updated Dictionary")
        
        return updated_df



UpdateDictionary('Data Cleaning',
                 description='',
                 command_code='',
                 classification='Theory',
                 category='ML',
                 sub_category='')




In [ ]:
Operational data is usually transactional data that is generated and stored by applications, often in a relational or non-relational database.
Analytical data is data that has been optimized for analysis and reporting, often in a data warehouse.
Data pipelines are used to orchestrate activities that transfer and transform data. Pipelines are the primary way in which data engineers implement repeatable extract, transform, and load (ETL) solutions that can be triggered based on a schedule or in response to events.
A data lake is a storage repository that holds large amounts of data in native, raw formats. Data lake stores are optimized for scaling to massive volumes (terabytes or petabytes) of data. The data typically comes from multiple heterogeneous sources, and may be structured, semi-structured, or unstructured
Azure Data Lake Storage Gen2 builds on blob storage and optimizes I/O of high-volume data by using a hierarchical namespace that organizes blob data into directories, and stores metadata about each directory and the files within it. This structure allows operations, such as directory renames and deletes, to be performed in a single atomic operation. Flat namespaces, by contrast, require several operations proportionate to the number of objects in the structure. Hierarchical namespaces keep the data organized, which yields better storage and retrieval performance for an analytical use case and lowers the cost of analysis.
In Azure Blob storage, you can store large amounts of unstructured ("object") data in a flat namespace within a blob container. Blob names can include "/" characters to organize blobs into virtual "folders", but in terms of blob manageability the blobs are stored as a single-level hierarchy in a flat namespace.

Descriptive analytics, which answers the question “What is happening in my business?”. The data to answer this question is typically answered through the creation of a data warehouse in which historical data is persisted in relational tables for multidimensional modeling and reporting.
Diagnostic analytics, which deals with answering the question “Why is it happening?”. This may involve exploring information that already exists in a data warehouse, but typically involves a wider search of your data estate to find more data to support this type of analysis.
Predictive analytics, which enables you to answer the question “What is likely to happen in the future based on previous trends and patterns?”
Prescriptive analytics, which enables autonomous decision making based on real-time or near real-time analysis of data, using predictive analytics.

Data virtualization allows you to interact with data without the need to understand how the data is formatted, structured, or what is its data type.

In [ ]:
BOLD - Build Together, Own it, Lead with Agility, Driven by Curiosity and Always Welcoming
Strategic Priorities - Integration, Business Transformation, Diversify Revenue, ESG

In [ ]:

RE has issue with Python escape clauses \, utilize r to make it string literal
r"\n"

* \b: Word boundary ensures that we match whole words.
* \w+: Matches one or more word characters (letters, digits, or underscores).
* \b: Another word boundary to ensure the end of the word.
* [^\w\s]: Matches any character that is not a word character (letters, digits, or underscores) or a whitespace character.
* |: OR operator.
* [\d]: Matches any digit.
* [^\w\s]: Matches any character that is not a word character (letters, digits, or underscores) or a whitespace character.
* |: OR operator.
* [\d]: Matches any digit.




Linux
Operating Systems are Layered.
Kernel is the inter most layer
Outermost layer is a GUI
Linux can peel off layers, helps towards simplicity
GUI can take  ~50% of processing power

Linux Distributions
GNU (GNU not Unix)
- Red Hat (CentOS)
- Fedora
Debian
- Ubuntu
- Raspberry Pi
- Various IoT
- 

Resources: 
- Command Line Help: www.explainshell.com


Linux Commands
man - Manual
History - history
Clear - clears history
Q - quits from current query
pwd- current directory
~ - short hand for home
. Current Directory
.. Parent Directory
Ls -l adds 6 additional fields 
ls -lh - adds file sizes
First letter - D: Directory, -: File, l - Link, Permission, Number of Hard links, Users who owns it. Size in bytes, date opened or last maintained  
Ls -la - shows hidden files
sudo = "super user (root) do"
mkdir - makes new directory
rmdir - removes directory
rm -r temp_1_3 - recursively removes directories
chmod 765 temp_1_1 - change permissions to directory


* r = read = 2^2 = 4
* w = write = 2^1 = 2
* x = execute = 2^0 = 1
* 1 = --x
* 2 = -w-
* 3 = -wx
* 4 = r--
* 5 = r-x
* 6 = rw-
* 7 = rwx

* u = user
* g = group
* o = other (warning: NOT owner!)

chmod u=rwx,g=rx,o=r *

ls -lhR - recursively list out directory tree
cp filename - copy
rm filename - remove
mv filename new_filename- rename

head returns the first few lines of a file; optionally pass the number of lines to return
tail returns the last few lines of a file; optionally pass the number of lines to return
grep matches a pattern in a file and returns lines that match the pattern
grep -v returns the opposite, the rows that do NOT match
| is a pipeline; Linux spawn a process for each command; Linux creates an "anonymous pipe" aka "un-named pipe" between the processes; Linux routes standard output to pipes and routes standard input from pipes for intermediate commands
diff finds the differences between two files
* vi filename
* Movement mode: arrow keys, page up, page down, 0 beginning of line, $ end of line, x delete character, dd delete line
* Movement mode to insert mode: i for insert, a for append, note that INSERT appears lower left
* Insert mode to movement mode: ESC key
* Movement mode to command mode: : note that hitting the enter key will run the command and return to movement mode
* Save changes: :w
* Save a file and exit: :wq
* Quit: :q
* Quit with unsaved changes: :q!


How to Change to Bash
chsh -s /bin/bash

In [2]:
from d_py_functions.Definitions import dictionary_definition

In [58]:
'Derek'.find('D')

0

In [56]:
from d_py_functions.SharedFolder import ReadDirectory

reference = ReadDirectory('//Users//derekdewald//Documents//Python//Github_Repo//Definitions')
reference


['Derek.csv']

In [22]:
import os
import pandas as pd
import numpy as np

os.getcwd()

'/Users/derekdewald/Documents/Python/Github_Repo'

In [73]:
def D_Dictionary(word,
                 folder='//Users//derekdewald//Documents//Python//Github_Repo//Definitions//',
                 dictionary_definition=dictionary_definition):
    
    '''
    
    '''

    try:
        df = pd.read_csv(f"{folder}{word}.csv")
    except:
        temp_dict = {}
        for key in dictionary_definition.keys():
            if key == 'Definition':
                pass
            else:
                try:
                    print(dictionary_definition[key])
                except:
                    pass
                
                temp_dict[key] = input(f"Please Select a {key}:")

        final_dict = {word:temp_dict}
        df = pd.DataFrame(final_dict).T.reset_index().rename(columns={'index':'Definition'})[list(dictionary_definition.keys())]
        df.to_csv(f"Definitions//{word}.csv",index=False)
        
    return df

a = D_Dictionary('Derek1')
a    

,Definition,Category,SubCategory,Notes,Links,SKlearn
0,Derek1,Test,Test,Test,test,Test


In [79]:
df = pd.read_excel('test.xlsx',sheet_name='Terminology')
df

,Unnamed: 0,Word,Category,Sub Categorization,Definition,Notes,Link
0,1.0,Accuracy,NaN,NaN,NaN,NaN,NaN
1,1.0,AdaBoost,Model,Emsemble,AdaBoost (Adaptive Boosting) is a machine lear...,NaN,NaN
2,1.0,ADASYN,NaN,NaN,Adaptive Synthetic Sampling: generates synthet...,NaN,NaN
3,1.0,Addition Rule,NaN,NaN,P ( AuB) = P(A) + P(B) - P(A intersection B).,NaN,NaN
4,1.0,Adjacency Matrix,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
170,NaN,Occam's Razor,Principle,NaN,Philosophical principle that suggests when fac...,NaN,NaN
171,NaN,Normal Distance,NaN,NaN,NaN,NaN,NaN
172,NaN,Euclidean Distance,NaN,NaN,NaN,NaN,NaN
173,NaN,Dot Product,NaN,NaN,NaN,NaN,NaN


In [69]:
pd.read_csv('Definitions/Derek1.csv')

,Definition,Category,SubCategory,Notes,Links,SKlearn
0,Derek1,Test,Test,Test,test,Test


# Plan

1) Streamlit
2) Hugging Face
3) Cloud Engine Certification
4) Automated Function to Download all google sheets somewhere

## Template

In [50]:
data_definitions['scikit_learn'] = {'Category':"",
                                    'SubCategory':"",
                                    'ModelType':"",
                                    'Definition':"",
                                    'Images':['scikit_learn_models.png']}

data_definitions['Boosting'] = {'Category':"",
                                'SubCategory':"",
                                'ModelType':"",
                                'Definition':"",
                                'Images':['Boosting_.png']}

data_definitions['Bagging'] = {'Category':"",
                                'SubCategory':"",
                                'ModelType':"",
                                'Definition':"",
                                'Images':['Bagging.png']}

data_definitions['Cosine Similiarity'] = {'Category':"",
                                          'SubCategory':"",
                                          'ModelType':"",
                                          'Definition':"",
                                          'Images':['cosine_similiarity.png']}

data_definitions['Kubernetes'] = {'Category':"",
                                  'SubCategory':"",
                                  'ModelType':"",
                                  'Definition':"",
                                  'Images':['Kubernetes.png']}

data_definitions['Anscombe Quartet'] = {'Category':"",
                                  'SubCategory':"",
                                  'ModelType':"",
                                  'Definition':"",
                                  'Images':['Anscombe_Quartet.png']}

data_definitions['Spark'] = {'Category':"",
                                  'SubCategory':"",
                                  'ModelType':"",
                                  'Definition':"",
                                  'Images':['spark.png']}


data_definitions['Harmonic Mean'] = {'Category':"",
                                  'SubCategory':"",
                                  'ModelType':"",
                                  'Definition':"",
                                  'Images':['harmonic_mean.png']}


data_definitions['X Or'] = {'Category':"",
                                  'SubCategory':"",
                                  'ModelType':"",
                                  'Definition':"",
                                  'Images':['X_OR.png']}

data_definitions['Sigmoid'] = {'Category':"",
                                  'SubCategory':"",
                                  'ModelType':"",
                                  'Definition':"",
                                  'Images':['Sigmoid.png']}

data_definitions['Loss Function'] = {'Category':"",
                                  'SubCategory':"",
                                  'ModelType':"",
                                  'Definition':"Need a systematic method to compare the effictivenes of multiple models, utilize a user defined function which measures for objective analysis",
                                  'Images':['Loss_Functions.png']}

data_definitions['Occams Razor'] = {'Category':"",
                        'SubCategory':"",
                        'ModelType':"",
                        'Definition':"All else equal, a simpler explanation is better.",
                        'Image':[]}


data_definitions['Variance Bias Trade Off'] = {'Category':"",
                                               'SubCategory':"",
                                               'ModelType':"",
                                               'Definition':"",
                                               'Image':['Variamce_Not_Equal.png','bias.png','bias1.png']}


data_definitions['Machine Learning'] = {'Category':"",
                                               'SubCategory':"",
                                               'ModelType':"",
                                               'Definition':"",
                                               'Image':['ML_AI_Significance_Chart.png','AI_ML_DL_Visual.png']}


data_definitions['Biparte Graph'] = {'Category':"",
                                               'SubCategory':"",
                                               'ModelType':"",
                                               'Definition':"",
                                               'Image':['Biparte_Graph.png']}

data_definitions['Logistic Regression'] = {'Category':"",
                                               'SubCategory':"",
                                               'ModelType':"",
                                               'Definition':"",
                                               'Image':['MultiClass_LogisticReg.png']}

data_definitions['Linear Regression'] = {'Category':"",
                                               'SubCategory':"",
                                               'ModelType':"",
                                               'Definition':"",
                                               'Image':['ImpactOutliers.png']}

data_definitions['MNist'] = {'Category':"",
                             'SubCategory':"",
                             'ModelType':"",
                             'Definition':"",
                             'Image':['ImpactOutliers.png','MNist_Results.png']}

data_definitions['Optimizers'] = {'Category':"",
                             'SubCategory':"",
                             'ModelType':"",
                             'Definition':"",
                             'Image':['Optimizers.png']}

data_definitions['Correlation'] = {'Category':"",
                                   'SubCategory':"",
                                   'ModelType':"",
                                   'Definition':"",
                                   'Image':['Correlation_Visual.png']}


data_definitions['Data Visualization'] = {'Category':"",
                                          'SubCategory':"",
                                          'ModelType':"",
                                          'Definition':"Allows people to offload cognition and processing from prefrontal cortex to visual receptors organized by spatial location. Serves as a high definition bandwidth to the brain. When creating a visualization, must consider what your intuitive goal and expectation is. Satify vs Optimize. Set of possiblilities: Known, Proposed, Solution. What data the user sees, why the user intends to use a vis tool, and how the visual encoding and interaction idioms are constructed on terms of design choice. Distinct Actions, Analyze, Search, Query",
                                          'Image':[]}


data_definitions['Hyperparameter Tuning'] = {'Category':"",
                                          'SubCategory':"",
                                          'ModelType':"",
                                          'Definition':"",
                                          'Image':[]}

data_definitions['Grid Search'] = {'Category':"Hyperparameter Tuning",
                                          'SubCategory':"",
                                          'ModelType':"",
                                          'Definition':"",
                                          'Image':[]}

data_definitions['Random Search'] = {'Category':"Hyperparameter Tuning",
                                          'SubCategory':"",
                                          'ModelType':"",
                                          'Definition':"",
                                          'Image':[]}

data_definitions['Bayesian'] = {'Category':"Hyperparameter Tuning",
                                          'SubCategory':"",
                                          'ModelType':"",
                                          'Definition':"",
                                          'Image':[]}

data_definitions['Tree Structured Parzen Estimator Search'] = {'Category':"Hyperparameter Tuning",
                                                               'SubCategory':"",
                                                               'ModelType':"",
                                                               'Definition':"",
                                                               'Image':[]}

data_definitions['Activation Functions'] = {'Category':"",
                                                               'SubCategory':"",
                                                               'ModelType':"",
                                                               'Definition':"",
                                                               'Image':[]}

data_definitions['Parametric'] = {'Category':"",
                                  'SubCategory':"",
                                  'ModelType':"",
                                  'Definition':"",
                                  'Image':[]}

data_definitions['Non Parametric'] = {'Category':"",
                                  'SubCategory':"",
                                  'ModelType':"",
                                  'Definition':"",
                                  'Image':[]}

data_definitions['Neural Networks'] = {'Category':"",
                                       'SubCategory':"",
                                       'ModelType':"",
                                       'Definition':"First Appeared in 2011",
                                       'Image':[]}

data_definitions['Softmax'] = {'Category':"",
                                       'SubCategory':"",
                                       'ModelType':"",
                                       'Definition':"",
                                       'Image':[]}

data_definitions['Entopy'] = {'Category':"",
                                       'SubCategory':"",
                                       'ModelType':"",
                                       'Definition':"",
                                       'Image':[]}


data_definitions['Ngram'] = {'Category':"",
                                       'SubCategory':"",
                                       'ModelType':"",
                                       'Definition':"",
                                       'Image':[]}



## Logistic Regression


### Model
$f(x) = w_0 + w_1X$

### Parameters 

$[w_0, w_1]$

### Loss
$J(w_0,w_1) = \frac{1}{n} \sum_{i=1}^n \left( y_i - (w_0 + w_1 x_i) \right)^2$

### Objective 
Minimize $J(w_0,w_1)$

### Mean Square Error
$MSE = \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2$

### Sigmoid Function

$\sigma(x) = \frac{1}{1 + e^{-x}}$


Linear Regression : $\hat{y} = xw^T + b$ <br>
Logistic Regression: $\hat{y} = \frac{1}{1 + e^({-xw^T + b})}$

In [43]:
## Decision Boundary

In [ ]:
## Cross Entropy

## Gradient Descent 

Predicitions: $\sigma(XW^T)$

Differences: $\sigma(XW^T) - Y$

Gradient: $\frac{1}{m} (\sigma(XW^T) -Y)$

$\text{Log Loss} = -\frac{1}{n} \sum_{i=1}^n \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]$


$\text{Log Loss} = -\frac{1}{n} \sum_{i=1}^n \sum_{j=1}^k y_{ij} \log(\hat{y}_{ij})$


In [60]:
## Softmax Normalization

$\sigma(z)_i = \frac{\sigma(xW^T_j)}{\sum_{j=1}^k \sigma(xW^T_j)}$

In [73]:
## Chain Rule

$h(x) = f(g(x))$


$h'(x) = f'(g(x)) \cdot g'(x)$


In [ ]:
## Boosting. Train Additive Models in Series Where Each Predicts the Residual from the Previous Model

In [ ]:
## Bagging. Trains Models in Parallel Via Bootstrap Sampling

## Distance Review

$ ||A|| = \sqrt{\sum \left (A_i \right)^2}$


$ L_2(A,B) = \sqrt{\sum \left( A_i - B_i \right)^2} $

$\ A \cdot B = \sum_{i=1}^n A_i B_i = ||A|| ||B|| cos(\theta)$


## Spark

## Kubernetes